In [30]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
dir_path = './' #local
# dir_path = './drive/MyDrive/BAKA/' #colab

In [40]:
#uncomment in colab
# from google.colab import drive
# drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '10' + '/' + '0.txt', header=None, sep=' ')
df.drop(df.index[100:])
filler = np.zeros((100-len(df), 31))
df = df.append(pd.DataFrame(filler), ignore_index=True )
df
# df = df.drop(columns=[31])
# sp_df = np.array_split(df, 2)
# sp_df[0]

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,164.280661,176.325903,178.121795,173.909634,179.942562,165.068138,179.880849,169.886642,179.693632,178.665290,...,0.589049,0.564965,-0.296812,0.587671,0.523166,-0.401421,0.608364,13.702051,7.694128,16.043218
1,165.122590,175.624965,179.600487,179.520481,177.638911,166.203109,168.936561,175.217537,179.756639,179.472871,...,0.589785,0.563702,-0.294720,0.586825,0.522370,-0.401031,0.608108,14.720415,4.666922,14.926564
2,165.078781,176.016382,179.983630,179.527066,178.258843,166.794220,169.728408,179.934052,179.742772,179.466085,...,0.588971,0.563148,-0.294356,0.586612,0.522276,-0.401130,0.608519,14.914579,3.442307,12.785423
3,160.916565,172.198336,179.873084,178.794688,171.190540,169.025568,169.618909,178.304716,179.278919,176.811213,...,0.588143,0.564566,-0.294643,0.588136,0.521309,-0.400198,0.607621,20.174759,4.338054,7.668797
4,159.852724,179.822872,179.118311,179.866069,178.201757,168.584125,175.381159,179.493500,178.231357,178.974019,...,0.585181,0.557927,-0.292103,0.584925,0.520796,-0.399540,0.607557,12.097098,5.726537,6.220917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
files = os.listdir(dir_path + 'DataCollection')
files

['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']

In [41]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[100:])
        if len(df) < 100:
            filler = np.zeros((100-len(df), 31))
            df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[100:])
        if len(df) < 100:
            filler = np.zeros((100-len(df), 31))
            df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

302   60
602   120
200   40
512   102
302   60
410   82
422   84
200   40
200   40
200   40
200   40


In [42]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (2842, 100, 31)
y_train.shiape:  (2842,)
x_test.shape:  (708, 100, 31)
y_test.shape:  (708,)
[[[150.587      144.419      135.792      ...  13.0169       7.71234
    11.0556    ]
  [150.859      144.337      135.95       ...  12.9418       7.68933
    11.0159    ]
  [150.946      144.576      136.628      ...  12.8984       7.67985
    10.8432    ]
  ...
  [153.648      144.694      138.125      ...  13.418        6.05023
    11.3359    ]
  [153.672      144.746      138.565      ...  13.3282       6.02335
    11.4155    ]
  [153.61       144.807      138.928      ...  13.2624       6.07887
    11.4544    ]]

 [[153.587      144.8        139.027      ...  13.2341       6.15317
    11.4522    ]
  [153.514      144.783      139.073      ...  13.2043       6.23986
    11.4108    ]
  [153.465      144.756      138.957      ...  13.1946       6.35819
    11.3776    ]
  ...
  [153.072      139.277      133.717      ...  13.7425       8.01148
    15.0987    ]
  [152.886      1

In [8]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [9]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [10]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2042, 100, 31)
y_train.shiape:  (2042,)
x_test.shape:  (508, 100, 31)
y_test.shape:  (508,)


In [11]:
model = Sequential()
model.add(LSTM(units=100, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(6, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 100)          52800     
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 100)          8

In [12]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=200,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/200
23/23 [==============================] - 42s 257ms/step - loss: 1.6793 - accuracy: 0.4241 - val_loss: 1.7454 - val_accuracy: 0.2146
Epoch 2/200
23/23 [==============================] - 5s 203ms/step - loss: 0.3826 - accuracy: 0.8845 - val_loss: 1.6815 - val_accuracy: 0.3171
Epoch 3/200
23/23 [==============================] - 5s 206ms/step - loss: 0.2610 - accuracy: 0.9316 - val_loss: 1.5989 - val_accuracy: 0.4244
Epoch 4/200
23/23 [==============================] - 5s 205ms/step - loss: 0.2113 - accuracy: 0.9483 - val_loss: 1.4961 - val_accuracy: 0.4780
Epoch 5/200
23/23 [==============================] - 5s 204ms/step - loss: 0.1627 - accuracy: 0.9574 - val_loss: 1.3390 - val_accuracy: 0.6098
Epoch 6/200
23/23 [==============================] - 5s 206ms/step - loss: 0.1468 - accuracy: 0.9604 - val_loss: 1.1434 - val_accuracy: 0.7317
Epoch 7/200
23/23 [==============================] - 5s 205ms/step - loss: 0.1377 - accuracy: 0.9669 - val_loss: 0.9546 - val_accuracy: 0.800

In [13]:
# model.save(dir_path + 'Models/gestures.h5')

In [14]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_100', save_format='tf')

Evaluate on test data
7/7 [==============================] - 1s 74ms/step - loss: 0.1182 - accuracy: 0.9724
test loss, test acc: [0.11823232471942902, 0.9724409580230713]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_100/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_100/assets


In [15]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [16]:
min_max_scaler.data_min_

array([ 9.30245e+01,  8.99999e+01,  9.00456e+01,  9.00455e+01,
        9.00455e+01,  6.54812e+01,  2.57840e+01,  2.77705e+01,
        2.79286e+01,  2.04904e+01, -4.24721e+02,  6.97324e+01,
       -2.97270e+02, -4.20951e+02,  5.95510e+01, -3.15328e+02,
       -4.48325e+02,  6.33254e+01, -3.17749e+02, -4.46258e+02,
        8.67906e+01, -3.13954e+02, -4.57886e+02,  8.58241e+01,
       -3.02162e+02, -4.42757e+02,  1.03148e+02, -2.82183e+02,
        1.86895e-01,  6.48663e-02,  1.07620e-01])